In [1]:
import time
from main.model.detect_odr import object_detection
from PIL import Image
import anki_vector
from anki_vector.util import degrees
import anki_vector.camera
from anki_vector.util import degrees, distance_mm, speed_mmps
import collections

C:\Users\suraj\anaconda3\envs\FasterRCNN\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\suraj\anaconda3\envs\FasterRCNN\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\suraj\anaconda3\envs\FasterRCNN\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\suraj\anaconda3\envs\FasterRCNN\lib\site-packages

Instructions for updating:
Use tf.gfile.GFile.


In [7]:
robot = anki_vector.Robot()
image_name = "detect.jpg"

23:31:33.455               robot.Robot WARNING  No serial number or name provided. Automatically selecting 00908dd7


In [8]:
def get_classnames(image_path):
    """
    This function calls the object detection library to detect 600 objects
    :param image_path:
    :return: class labels
    """
    try:
        classes = object_detection(image_path)
        if len(classes) == 0:
            return 'no objects'
        class_list = []
        for class_names in classes:
           class_list.append(class_names)
        return ', '.join(class_list)

    except:
        return 'no objects'


In [20]:
# Create a deque to store last three distance values to deal with random noises
deq = collections.deque([15, 15, 15],3)

In [ ]:
with anki_vector.Robot() as robot:
    robot.motors.set_head_motor(-5.0)
    while True:
        # Get the distance value of the sensor reading
        distance = robot.proximity.last_sensor_reading.distance
        avg = sum(deq)/3
        # Avg value in 
        print('Distance values-Avg(DEQUE):',avg)
        robot.motors.set_wheel_motors(50, 50)
        
        # if Vector is close to obstacle      
        if 2 < avg <= 3.5:
            # Stop the motors
            robot.motors.stop_all_motors()
            robot.behavior.say_text('obstacle detected, Wait a moment')
            robot.camera.init_camera_feed()
            robot.vision.enable_display_camera_feed_on_face(True)
            robot.camera.latest_image.raw_image.save(image_name, 'JPEG')
           
            # Comment the Next two lines if you dont want to call object detector
            # Also, if you dont have GPU the next two steps will make the vector slow
            text = get_classnames(image_name)
            robot.behavior.say_text('I can detect {}'.format(text))
              
             # You are free to modify the motor values.
            robot.behavior.turn_in_place(degrees(-70))
            robot.behavior.drive_straight(distance_mm(120), speed_mmps(80))
            robot.behavior.turn_in_place(degrees(70))


23:47:41.998               robot.Robot WARNING  No serial number or name provided. Automatically selecting 00908dd7
23:47:42.002     connection.Connection    INFO  Connecting to 192.168.137.64:443 for Vector-Y1Y2 using C:\Users\suraj\.anki_vector\Vector-Y1Y2-00908dd7.cert
23:47:42.526     connection.Connection    INFO  BehaviorControl control_granted_response { }


Distance values-Avg(DEQUE): 3.3070866141732282
